In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import xgboost as xgb

In [ ]:
df = pd.read_csv("train.csv")

df = df.rename(columns={"length(m)": "length", "height(cm)": "height"})

In [ ]:
X = df.drop(['breed_category','pet_category', "pet_id"], axis=1)

temp = X["listing_date"]
hh = []
for i in temp:
  time = i.split()
  hours = time[1]
  hour = hours.split(":")
  if int(hour[1]) < 30:
    hh.append(int(hour[0]))
  elif int(hour[1]) >=30 and int(hour[0]) < 23:
    hh.append(int(hour[0])+1)
  elif int(hour[1]) >=30 and int(hour[0]) == 23:
    hh.append(0)

X["hours"] = hh

In [ ]:
X["issue_date"] = pd.to_datetime(X["issue_date"])
X["listing_date"] = pd.to_datetime(X["listing_date"])
X["duration"] = (X["listing_date"] - X["issue_date"]).dt.days

enc1 = OneHotEncoder()
x = pd.DataFrame(enc1.fit_transform(X[['color_type']]).toarray())
X = X.join(x)

X["condition"].fillna(3, inplace=True)
X['condition'] = X['condition'].astype(int)

X = X.drop(columns=["issue_date", "listing_date","color_type"])

In [ ]:
x = ["length","height","X1","X2"]
scaler = preprocessing.StandardScaler()
scaled_X = scaler.fit_transform(X[x])
X[x] = pd.DataFrame(scaled_X, columns=x)

In [ ]:
df2 = pd.read_csv("test.csv")

df2 = df2.rename(columns={"length(m)": "length", "height(cm)": "height"})

In [ ]:
X2 = df2.drop(["pet_id"], axis=1)

temp = X2["listing_date"]
hh = []
for i in temp:
  time = i.split()
  hours = time[1]
  hour = hours.split(":")
  if int(hour[1]) < 30:
    hh.append(int(hour[0]))
  elif int(hour[1]) >=30 and int(hour[0]) < 23:
    hh.append(int(hour[0])+1)
  elif int(hour[1]) >=30 and int(hour[0]) == 23:
    hh.append(0)

X2["hours"] = hh

In [ ]:

X2["issue_date"] = pd.to_datetime(X2["issue_date"])
X2["listing_date"] = pd.to_datetime(X2["listing_date"])
X2["duration"] = (X2["listing_date"] - X2["issue_date"]).dt.days

enc2 = OneHotEncoder()
x = pd.DataFrame(enc2.fit_transform(X2[['color_type']]).toarray())
X2 = X2.join(x)

X2["condition"].fillna(3, inplace=True)
X2['condition'] = X2['condition'].astype(int)

X2 = X2.drop(columns=["issue_date", "listing_date","color_type"])

In [ ]:
ll1, ll2, ll3 = [], [], []
for i in enc1.categories_:
  for j in i:
    ll1.append(j)

for i in enc2.categories_:
  for j in i:
    ll2.append(j)

for i in ll1:
  if i not in ll2:
    ll3.append(i)
    
X = X.drop(columns=[6,19])

In [ ]:
x = ["length","height","X1","X2"]

scaler = preprocessing.StandardScaler()
scaled_X = scaler.fit_transform(X2[x])
X2[x] = pd.DataFrame(scaled_X, columns=x)

In [ ]:
def range_part(column,value):
    if value > info.loc[4,column]:
        return 5
    elif value > info.loc[2,column]:
        return 4
    elif value > info.loc[1,column]:
        return 3
    elif value > info.loc[0,column]:
        return 2
    elif value > info.loc[3,column]:
        return 1
    else:
        return 0

In [ ]:
info = pd.DataFrame()
info['length'] = [np.percentile(X['length'],25*i) for i in range(1,4)]
info['height'] = [np.percentile(X['height'],25*i) for i in range(1,4)]
info['duration'] = [np.percentile(X['duration'],25*i) for i in range(1,4)]
info.loc[3] = [2.5*info.loc[0,column] - 1.5*info.loc[2,column] for column in info.columns]
info.loc[4] = [2.5*info.loc[2,column] - 1.5*info.loc[0,column] for column in info.columns]

In [ ]:
from tqdm import tqdm
tqdm.pandas()
X['length_range'] = X['length'].progress_apply(lambda x:range_part('length',x))
X['height_range'] = X['height'].progress_apply(lambda x:range_part('height',x))
X['duration_range'] = X['duration'].progress_apply(lambda x:range_part('duration',x))

X2['length_range'] = X2['length'].progress_apply(lambda x:range_part('length',x))
X2['height_range'] = X2['height'].progress_apply(lambda x:range_part('height',x))
X2['duration_range'] = X2['duration'].progress_apply(lambda x:range_part('duration',x))

In [ ]:
y1 = df["breed_category"]
y2 = df["pet_category"]

y = df[["breed_category", "pet_category"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

y_train1 = y_train["breed_category"]
y_train2 = y_train["pet_category"]

y_test1 = y_test["breed_category"]
y_test2 = y_test["pet_category"]

In [ ]:
corr = X.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(X.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(X.columns)
ax.set_yticklabels(X.columns)
plt.show()

#SVC

In [ ]:
clf1 = SVC(C=10,gamma=10, kernel="rbf")
ypred1 = clf1.predict(X)
s1 = f1_score(y1, ypred1, average="weighted")

In [ ]:
clf2 = SVC(C=10,gamma=10, kernel="rbf").fit(X, y2)
ypred2 = clf2.predict(X)
s2 = f1_score(y2, ypred2, average="weighted")

In [ ]:
scores = cross_val_score(clf1, X, y2, cv=5, scoring="f1_weighted")
print(scores)

score = 100 * (s1+s2) / 2
print(s1,s2)
print("Training f1 score: ", score)

In [ ]:
# ypred1 = clf1.predict(X_train)
# s1 = f1_score(y_train1, ypred1, average="weighted")
# ypred2 = clf2.predict(X_train)
# s2 = f1_score(y_train2, ypred2, average="weighted")
0.9138437651219301 0.8973397476796859
Training f1 score:  90.55917564008081

In [ ]:
X2 = df2.drop(columns=["pet_id"])
ypred11 = clf1.predict(X2)
ypred22 = clf2.predict(X2)

In [ ]:
submission = pd.DataFrame({"pet_id": df2["pet_id"], "breed_category": ypred11, "pet_category": ypred22})
print(submission.shape)
submission.to_csv("submission.csv", index=False)

#XGBoost

In [ ]:
# D_train1 = xgb.DMatrix(X, label=y1)
# D_train2 = xgb.DMatrix(X, label=y2)
# D_test1 = xgb.DMatrix(X_test, label=y_test1)
# D_test2 = xgb.DMatrix(X_test, label=y_test2)

# param1 = {
#     'eta': 0.3, 
#     'max_depth': 6,
#     "gamma": 0.1,
#     'objective': 'multi:softmax',
#     "nthread": 5,
#     # "lambda": 20,
#     # "alpha": 0.1,
#     "eval_metric": "mlogloss",
#     'num_class': 5} 

# param2 = {
#     'eta': 0.3, 
#     'max_depth': 6,
#     "gamma": 0.01,  
#     # "lambda": 10,
#     # "alpha": 0.1,
#     'objective': 'multi:softmax',
#     "eval_metric": "mlogloss",  
#     'num_class': 5} 

# steps = 20 

# model1 = xgb.train(param1, D_train1, steps)
# model2 = xgb.train(param2, D_train2, steps)

model1 = xgb.XGBClassifier(learning_rate=0.1,  
                      objective='multi:softmax', 
                      n_estimators=1000,
                      gamma=10,
                      max_depth=6).fit(X,y1) 

model2 = xgb.XGBClassifier(learning_rate=0.1,  
                      objective='multi:softmax', 
                      n_estimators=1000,
                      gamma=10, 
                      max_depth=6).fit(X,y2)

In [ ]:
train_preds1 = model1.predict(X)
train_preds2 = model2.predict(X)

s1 = f1_score(y1, train_preds1, average="weighted")
s2 = f1_score(y2, train_preds2, average="weighted")
score = 100 * (s1+s2) / 2
print(s1,s2)
print("Training f1 score: ", score)

In [ ]:
pd.DataFrame(preds1, columns=['Value'])['Value'].value_counts().plot(kind='barh')

In [ ]:
test_preds1 = model1.predict(X2)
test_preds2 = model2.predict(X2)

# s1 = f1_score(y_test1, preds1, average="weighted")
# s2 = f1_score(y_test2, preds2, average="weighted")
# score = 100 * (s1+s2) / 2
# print(s1,s2)
# print("Test f1 score: ", score)

# pd.DataFrame(test_preds1, columns=['Value'])['Value'].value_counts().plot(kind='barh')

In [ ]:
# D_train3 = xgb.DMatrix(X2)

ypred11 = model1.predict(X2)
ypred22 = model2.predict(X2)

In [ ]:
submission = pd.DataFrame({"pet_id": df2["pet_id"], "breed_category": ypred11, "pet_category": ypred22})
print(submission.shape)
submission.to_csv("submission.csv", index=False)

#LGBM

In [44]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint

lgb1 = LGBMClassifier(
    reg_lambda=57,
    refit=True,
    # reg_alpha=10,
    # learning_rate=0.1,
    max_depth=-1,
    n_estimators=200).fit(X,y1)

lgb2 = LGBMClassifier(
    # reg_lambda=0.1,
    # reg_alpha=0.01,
    learning_rate=0.1,
    refit=True,
    n_estimators=200).fit(X,y2)

# param_test ={'num_leaves': sp_randint(6, 50), 
#              'min_child_samples': sp_randint(100, 500), 
#              'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#              'subsample': sp_uniform(loc=0.2, scale=0.8), 
#              'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
#              'reg_alpha': [40,42,44,56,57, 50, 60, 55],
#              'reg_lambda': [10,12,13, 15, 20,25],
#               'max_depth': [1,2,3,4,5,10,-1]}

# clf1 = RandomizedSearchCV(lgb1, param_test, random_state=0)
# clf2 = RandomizedSearchCV(lgb2, param_test, random_state=0)

# search1 = clf1.fit(X,y1)
# search2 = clf2.fit(X,y2)

# print(search1.best_params_)
# print(search2.best_params_)

In [45]:
train_preds1 = lgb1.predict(X)
train_preds2 = lgb2.predict(X)

s1 = f1_score(y1, train_preds1, average="weighted")
s2 = f1_score(y2, train_preds2, average="weighted")
score = 100 * (s1+s2) / 2
print(s1,s2)
print("Training f1 score: ", score)

0.9582441309147597 0.9442032675688672
Training f1 score:  95.12236992418136


In [46]:
ypred11 = lgb1.predict(X2)
ypred22 = lgb2.predict(X2)

In [47]:
submission = pd.DataFrame({"pet_id": df2["pet_id"], "breed_category": ypred11, "pet_category": ypred22})
print(submission.shape)
submission.to_csv("submission3.csv", index=False)

(8072, 3)


#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model1 = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0).fit(X, y1)

model2 = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0).fit(X, y2)

In [ ]:
train_preds1 = model1.predict(X)
train_preds2 = model2.predict(X)

s1 = f1_score(y1, train_preds1, average="weighted")
s2 = f1_score(y2, train_preds2, average="weighted")
score = 100 * (s1+s2) / 2
print(s1,s2)
print("Training f1 score: ", score)

In [ ]:
ypred11 = model1.predict(X2)
ypred22 = model2.predict(X2)

In [ ]:
submission = pd.DataFrame({"pet_id": df2["pet_id"], "breed_category": ypred11, "pet_category": ypred22})
print(submission.shape)
submission.to_csv("submission10.csv", index=False)